In [ ]:
from tqdm import tqdm
import math
import pandas as pd

# Data Loading

In [ ]:
def load_bigrams(path):

    file = open(path, 'r')

    bigrams = {}
    all_oc = 0
    
    for line in tqdm(file):

        oc, word_1, word_2 = line.split()
        oc = int(oc)

        if int(oc) < 5: continue

        if word_2[-1] == "\n": word_2 = word2[:-1]
        all_oc += oc

        if (word_1, word_2) in bigrams.keys():
            bigrams[(word_1, word_2)] += oc
        else:
            bigrams[(word_1, word_2)] = oc

    return bigrams, all_oc

In [ ]:
def load_supertags(path):

    file = open(path, 'r')
    word_to_tag = {}

    for line in tqdm(file):

        word, tag = line.split(" ")
        if tag[-1] == "\n": tag = tag[:-1]

        word_to_tag[word.lower()] = tag
    
    file.close()
    return word_to_tag

In [ ]:
def load_unigrams(path):

    file = open(path, 'r')
    word_to_occurrence = {}
    all_oc = 0

    for line in tqdm(file):

        words = line.lower().split(" ")
        
        for word in words:

            all_oc += 1
            if word[-1] == "\n": word = word[:-1]

            if word in word_to_occurrence.keys():
                word_to_occurrence[word] += 1
            else:
                word_to_occurrence[word] = 1

    return word_to_occurrence, all_oc

In [ ]:
def supertags_stats(unigrams, bigrams, word_to_tag):

    unitags = {}
    unitags_all_oc = 0
    bitags = {}
    bitags_all_oc = 0

    for unigram in unigrams.keys():

        if unigram in word_to_tag.keys():
            unitags_all_oc += unigrams[unigram]
            if word_to_tag[unigram] in unitags.keys():
                unitags[word_to_tag[unigram]] += unigrams[unigram] 
            else:
                unitags[word_to_tag[unigram]] = unigrams[unigram]

    for bigram in bigrams.keys():
        
        if bigram[0] in word_to_tag.keys() and bigram[1] in word_to_tag.keys():
             bitags_all_oc += bigrams[bigram]

             tag_1 = word_to_tag[bigram[0]]
             tag_2 = word_to_tag[bigram[1]]

             if (tag_1, tag_2) in bitags.keys():
                bitags[(tag_1, tag_2)] += bigrams[bigram]
             else:
                bitags[(tag_1, tag_2)] = bigrams[bigram]

    return unitags, unitags_all_oc, bitags, bitags_all_oc

In [ ]:
bigrams, bigrams_all_oc = load_bigrams('/content/poleval_2grams.txt')
unigrams, unigrams_all_oc = load_unigrams('/content/polish_corpora.txt')
word_to_tag = load_supertags('/content/drive/My Drive/Colab Notebooks/NLP/Dane/Copy of supertags.txt')

59134224it [01:11, 827856.05it/s]
23011601it [05:20, 71758.68it/s]
1781994it [00:04, 359139.36it/s]


In [ ]:
unitags, unitags_all_oc, bitags, bitags_all_oc = supertags_stats(unigrams, bigrams, word_to_tag)

# Colocation testing

In [ ]:
def pmi(word_1, word_2):

    oc = bigrams[(word_1, word_2)]

    res = 0
    if word_1 in unigrams.keys() and word_2 in unigrams.keys():
        res = math.log((oc/bigrams_all_oc)/((unigrams[word_1]/unigrams_all_oc)*
                                          (unigrams[word_2]/unigrams_all_oc)))
    return res

In [ ]:
def psm(word_1, word_2):

    oc = bigrams[(word_1, word_2)]

    res = 0
    if word_1 in unigrams.keys() and word_2 in unigrams.keys():
        f_xy = oc
        fhat_xy = unigrams[word_1]*unigrams[word_2]/unigrams_all_oc
        res = f_xy * (math.log(f_xy) - fhat_xy - 1)
    
    return res

In [ ]:
def t_student(word_1, word_2):

    oc = bigrams[(word_1, word_2)]

    res = 0
    if word_1 in unigrams.keys() and word_2 in unigrams.keys():
        f_xy = oc/bigrams_all_oc 
        fhat_xy = ((unigrams[word_1]/unigrams_all_oc) * (unigrams[word_2]/unigrams_all_oc))
    
        res = (f_xy - fhat_xy) / math.sqrt(f_xy)
    
    return res

In [ ]:
def pmiwords_and_pmitags(word_1, word_2):

    pmi_words = pmi(word_1, word_2)

    pmi_tags = 0
    if word_1 in word_to_tag.keys() and word_2 in word_to_tag.keys():

        tag_1 = word_to_tag[word_1]
        tag_2 = word_to_tag[word_2]

        oc = bitags[(tag_1, tag_2)]

        pmi_tags = (oc/bitags_all_oc)/((unitags[tag_1]/unitags_all_oc)*
                                            (unitags[tag_2]/unitags_all_oc))
        if pmi_tags != 0:
            pmi_tags = math.log(pmi_tags)

    return pmi_words + max(0, pmi_tags)

In [ ]:
def best_col(f, word):

    candidates = [(word1, word2) for word1, word2 in bigrams.keys() if word1==word or word2==word]
    results = {}

    for candidate in candidates:

        results[candidate] = f(candidate[0],candidate[1]) 

    return sorted(results.items(), key=lambda x: x[1], reverse=True)

In [ ]:
words = ['koń', 'ojciec', 'dom', 'poduszka', 'czerwony', 'pisać', 'koszula', 'kot', 'kobieta', 'pies']
pd.set_option("display.max_rows", None, "display.max_columns", None)

for word in words:

    best_pmi = best_col(pmi, word)[:7]
    best_psm = best_col(psm, word)[:7]
    best_t_student = best_col(t_student, word)[:7]
    best_pmitags = best_col(pmiwords_and_pmitags, word)[:7]


    multicol_pmi = pd.MultiIndex.from_tuples([('PMI', 'Points'), ('PMI', 'Pair')])
    multicol_psm = pd.MultiIndex.from_tuples([('PSM', 'Points'), ('PSM', 'Pair')])
    multicol_pmitags = pd.MultiIndex.from_tuples([('PMITAGS', 'Points'), ('PMITAGS', 'Pair')])
    multicol_t_student = pd.MultiIndex.from_tuples([('T_student', 'Points'), ('T_student', 'Pair')])

    print('------------------------------------------ {} ------------------------------------------'.format(word))
    display(pd.concat([pd.DataFrame(best_pmi, columns=multicol_pmi), pd.DataFrame(best_psm, columns=multicol_psm), 
                     pd.DataFrame(best_pmitags, columns=multicol_pmitags), pd.DataFrame(best_t_student, columns=multicol_t_student)], axis = 1))


------------------------------------------ koń ------------------------------------------


PMI                          PSM              \
                 Points       Pair            Points        Pair   
0  (koń, gorącokrwisty)  12.277124  (koń, trojański)  122.489501   
1     (koń, wierzchowy)  12.094802     (czarny, koń)   87.814574   
2      (koń, trojański)  11.890008  (koń, pociągowy)   78.895153   
3      (koń, pociągowy)  11.822868    (szalony, koń)   52.190982   
4     (koń, zaprzęgowy)  11.283872      (biały, koń)   45.030526   
5   (koń, zimnokrwisty)  11.283872         (koń, ca)   39.697251   
6  (koń, przewalskiego)  10.667686         (koń, cu)   33.983850   

                PMITAGS                    T_student            
                 Points       Pair            Points      Pair  
0  (koń, gorącokrwisty)  14.452644        (jak, koń)  0.000420  
1     (koń, wierzchowy)  14.270323          (koń, ,)  0.000378  
2      (koń, trojański)  14.065528         (że, koń)  0.000345  
3      (koń, pociągowy)  13.998389  (koń, trojański)  0.000330  
4     (koń, zaprzęgowy)  13.459393       (koń, jest)  0.000316  
5   (koń, zimnokrwisty)  13.459393     (czarny, koń)  0.000294  
6  (koń, przewalskiego)  10.667686        (koń, nie)  0.000282

------------------------------------------ ojciec ------------------------------------------


PMI                            PSM               \
                Points      Pair               Points         Pair   
0  (ojciec, chrzestny)  9.792649     (ojciec, święty)  3665.575348   
1    (ojciec, rutilio)  9.718055        (mój, ojciec)  1547.017275   
2   (ojciec, soborowy)  8.910963  (ojciec, chrzestny)   915.462059   
3  (przybrany, ojciec)  8.611537      (ojciec, zmarł)   896.489159   
4   (ojciec, duchesne)  8.514082        (bóg, ojciec)   627.760588   
5     (ojciec, goriot)  8.268959   (ojciec, pracował)   485.139847   
6   (ojciec, laurenty)  8.218618     (ojciec, zginął)   429.526571   

               PMITAGS                    T_student            
                Points       Pair            Points      Pair  
0    (ojciec, lubecki)  12.078753    (jego, ojciec)  0.003940  
1  (ojciec, chrzestny)  11.299240     (ojciec, był)  0.002180  
2     (ojciec, święty)  11.228311     (jej, ojciec)  0.002178  
3     (ojciec, medard)  11.163000       (ojciec, ,)  0.001500  
4   (ojciec, soborowy)  10.417555  (ojciec, święty)  0.001316  
5  (ojciec, honoriusz)  10.216792     (mój, ojciec)  0.001064  
6   (ojciec, meissner)   9.864775       (a, ojciec)  0.000970

------------------------------------------ dom ------------------------------------------


...

PMI                         PSM              \
              Points       Pair           Points        Pair   
0    (kot, napłakał)  11.806221    (maciej, kot)  701.815382   
1    (gadający, kot)  10.417657    (czarny, kot)  321.696893   
2    (kot, szczekał)   9.893586      (rasy, kot)  163.233366   
3     (kot, filemon)   9.653051       (mój, kot)  148.688682   
4   (kot, bengalski)   9.112332  (kot, napłakał)  141.679178   
5        (kot, prot)   9.108324    (tomasz, kot)   99.493961   
6  (kot, syberyjski)   8.689613      (twój, kot)   93.644808   

             PMITAGS                 T_student            
              Points       Pair         Points      Pair  
0    (kot, napłakał)  13.577696       (kot, ,)  0.000725  
1   (kot, bengalski)  11.105265  (maciej, kot)  0.000652  
2       (dziki, kot)  11.043098  (czarny, kot)  0.000479  
3  (kot, syberyjski)  10.682547     (jak, kot)  0.000464  
4   (kot, sylwester)  10.530208       (kot, ()  0.000382  
5    (gadający, kot)  10.417657    (rasy, kot)  0.000369  
6    (kot, szczekał)   9.893586     (mój, kot)  0.000364

------------------------------------------ kobieta ------------------------------------------


PMI                            PSM               \
                 Points      Pair               Points         Pair   
0  (kobieta, 65-letnia)  9.986363     (każda, kobieta)  2837.072662   
1  (54-letnia, kobieta)  9.649891     (młoda, kobieta)  1673.886380   
2    (sposora, kobieta)  9.620904  (pierwsza, kobieta)  1174.354396   
3   (ciezarna, kobieta)  9.467570     (jedna, kobieta)   505.229969   
4  (38-letnia, kobieta)  9.387527    (piękna, kobieta)   349.250569   
5  (50-letnia, kobieta)  9.179887   (starsza, kobieta)   311.492062   
6    (zamozna, kobieta)  9.179887   (kobieta, powinna)   309.295204   

                      PMITAGS                       T_student            
                       Points       Pair               Points      Pair  
0    (kobieta, spodziewająca)  11.941959         (kobieta, ,)  0.001537  
1         (kobieta, ciężarna)  11.829309        (że, kobieta)  0.001296  
2         (kobieta, karmiąca)  11.370555     (każda, kobieta)  0.001273  
3  (przedsiębiorcza, kobieta)  11.349177  (pierwsza, kobieta)  0.000948  
4      (czarnoskóra, kobieta)  11.161551     (młoda, kobieta)  0.000938  
5       (demoniczna, kobieta)  11.088451       (kobieta, nie)  0.000926  
6         (ciężarna, kobieta)  11.006518        (kobieta, ma)  0.000851

------------------------------------------ pies ------------------------------------------


PMI                            PSM              \
                Points       Pair              Points        Pair   
0    (pies, descalzos)  11.374109         (mój, pies)  366.559152   
1    (berneński, pies)  10.712711        (twój, pies)  298.180663   
2   (pies, stróżujący)  10.589154  (pies, pogrzebany)  257.103169   
3    (bezpański, pies)  10.457818        (pies, rasy)  149.917169   
4   (pies, asystujący)  10.433126   (pies, myśliwski)  148.900216   
5  (pies, huckleberry)  10.121346   (pies, pasterski)  145.472080   
6     (pies, tropiący)   9.987815       (każdy, pies)  129.847856   

              PMITAGS                T_student            
               Points       Pair        Points      Pair  
0     (dorosły, pies)  12.763019  (pies, jest)  0.000673  
1   (pies, pasterski)  12.158348    (że, pies)  0.000667  
2   (berneński, pies)  12.104570   (jak, pies)  0.000658  
3   (bezpański, pies)  11.849678    (to, pies)  0.000637  
4  (pies, kanaryjski)  11.521482   (pies, nie)  0.000573  
5  (pies, stróżujący)  11.489447     (pies, ,)  0.000572  
6   (pies, descalzos)  11.374109   (mój, pies)  0.000522